# 创建 Deployment

让我们创建一个 `task_maistro` 应用的 deployment，这是我们在 module 5 中创建的应用。

## 代码结构

要创建 LangGraph Platform deployment，[需要提供以下信息](https://langchain-ai.github.io/langgraph/concepts/application_structure/)：

* 一个 [LangGraph API 配置文件](https://langchain-ai.github.io/langgraph/concepts/application_structure/#configuration-file) - `langgraph.json`
* 实现应用逻辑的 graphs - 例如 `task_maistro.py`
* 指定运行应用所需依赖的文件 - `requirements.txt`
* 提供应用运行所需的环境变量 - `.env` 或 `docker-compose.yml`

我们已经在 `module-6/deployment` 目录中准备好了这些文件！

## CLI

[LangGraph CLI](https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/) 是用于创建 LangGraph Platform deployment 的命令行工具。

In [ ]:
%%capture --no-stderr
# ================== 安装 LangGraph CLI ==================

# LangGraph CLI 知识点:
# - LangGraph CLI 是用于管理 LangGraph Platform deployments 的命令行工具
# - 提供以下核心功能:
#   1. langgraph build: 构建 Docker 镜像
#   2. langgraph dev: 本地开发服务器
#   3. langgraph up: 启动 deployment
#
# 为什么需要 CLI:
# - 简化 deployment 流程
# - 自动化 Docker 镜像构建
# - 统一的开发和生产环境管理
#
# 安装说明:
# - -U flag: 升级到最新版本
# - langgraph-cli: CLI 工具包名称

%pip install -U langgraph-cli

要创建[自托管 deployment](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#how-to-do-a-self-hosted-deployment-of-langgraph)，我们需要遵循以下几个步骤。

### 为 LangGraph Server 构建 Docker 镜像

首先，我们使用 langgraph CLI 为 [LangGraph Server](https://docs.google.com/presentation/d/18MwIaNR2m4Oba6roK_2VQcBE_8Jq_SI7VHTXJdl7raU/edit#slide=id.g313fb160676_0_32) 创建一个 Docker 镜像。

这将把我们的 graph 和依赖项打包到一个 Docker 镜像中。

Docker 镜像是 Docker 容器的模板，包含运行应用所需的代码和依赖项。

确保已安装 [Docker](https://docs.docker.com/engine/install/)，然后运行以下命令创建 Docker 镜像 `my-image`：

```
$ cd module-6/deployment
$ langgraph build -t my-image
```

### 设置 Redis 和 PostgreSQL

如果你已经有运行中的 Redis 和 PostgreSQL（例如，在本地或其他服务器上），那么可以[单独](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#running-the-application-locally)创建并运行 LangGraph Server 容器，并提供 Redis 和 PostgreSQL 的 URIs：

```
docker run \
    --env-file .env \
    -p 8123:8000 \
    -e REDIS_URI="foo" \
    -e DATABASE_URI="bar" \
    -e LANGSMITH_API_KEY="baz" \
    my-image
```

或者，你可以使用提供的 `docker-compose.yml` 文件，基于定义的服务创建三个独立的容器：

* `langgraph-redis`: 使用官方 Redis 镜像创建新容器
* `langgraph-postgres`: 使用官方 Postgres 镜像创建新容器
* `langgraph-api`: 使用你预构建的镜像创建新容器

只需复制 `docker-compose-example.yml` 并添加以下环境变量来运行部署的 `task_maistro` 应用：

* `IMAGE_NAME`（例如 `my-image`）
* `LANGSMITH_API_KEY`
* `OPENAI_API_KEY`

然后，[启动 deployment](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#using-docker-compose)：

```
$ cd module-6/deployment
$ docker compose up
```